# Modules

In [1]:
import math
import random
#import re
from functools import reduce
from itertools import permutations, combinations
# import pandas as pd
#import sympy
#sympy.init_printing()

import sys; sys.path.append("../modules")
from Permutation import *

from tqdm.notebook import tqdm
#from multiprocessing import Pool, cpu_count
from multiprocess import Pool

# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()

from knot_floer_homology import *
#help(pd_to_hfk)

In [ ]:
N = 4

def cross(x: int, y: int, sign: int):
    if sign not in range(4): raise(ArgumentError) # signs are 0b00, 0b01, 0b10, 0b11
    if (sign & 0b10):
        ft, lt = (x, x%(2*N)+1), (y, y%(2*N)+1)
    else:
        ft, lt = (y, y%(2*N)+1), (x, x%(2*N)+1)
    if (sign & 0b01):
        pd = (ft[0], lt[0], ft[1], lt[1])
    else:
        pd = (ft[0], lt[1], ft[1], lt[0])
    return pd    

lmd_data = [0 for _ in range(10)]
def calc_hfk(signs):
    print(bin(signs))
    for perm in permutations(range(1, N+1), N):
        if (perm[0] == N) or (perm[N-1] == 1):
            return None
        #-----
        pd = [cross(N+1+i, perm[i], ((signs >> 2*i) & 0b11)) for i in range(N)]
        #---
        cs = [Cycle([x-1 for x in tup], 2*N) for tup in pd]
        pd_perm = reduce((lambda x,y: x*y), cs) #; print(f"{pd_perm}\n\n")
        num_of_orbits = len(cycle_decomp(pd_perm))
        print(f"{perm} --> {pd_perm.image} --> {num_of_orbits}\n")
        if num_of_orbits - N == 2:
            print(pd)
            hfk = pd_to_hfk(pd)
            if hfk['fibered'] and hfk['seifert_genus'] > 0:
                sg = hfk['seifert_genus']
                ranks = hfk['ranks']
                for pg in ranks.keys():
                    if pg[0] == sg:
                        lmd = pg[1]
                print(lmd)
                lmd_data.setdefault(str(lmd), 0) 
                lmd_data[str(lmd)] += 1
                if lmd < 0:
                    print(pd)
                    return pd
        return None
for signs in range(4**(N-1)):
    calc_hfk(signs)
# with Pool(processes=8) as pool:
#     list(tqdm(pool.imap(calc_hfk, range(4**(N-1))), total = 4**(N-1)))
# #print(len(output), math.factorial(N)*(4**(N-1)))
print(lmd_data)

## scratch

In [ ]:
mdic = {'a': 4}

mdic.setdefault('a', 0)
mdic['a'] += 1
print(mdic)

In [ ]:
N = 6
hfk_data = []

def cross(x: int, y: int, sign: int):
    if sign not in range(4): raise(ArgumentError) # signs are 0b00, 0b01, 0b10, 0b11
    if (sign & 0b10):
        ft, lt = (x, x%(2*N)+1), (y, y%(2*N)+1)
    else:
        ft, lt = (y, y%(2*N)+1), (x, x%(2*N)+1)
    #---
    pd = (ft[0], lt[0], ft[1], lt[1]) if (sign & 0b01) else (ft[0], lt[1], ft[1], lt[0])
    return pd    

for perm in tqdm(permutations(range(1,N+1), N), total=math.factorial(N)):
#for perm in permutations(range(1,N+1), N): # iteratation on permutations on {N+1, ... ,2*N+1} 
    if (perm[0] != N) and (perm[N-1] != 1):
        for signs in range(4**(N-1)):
            pd = [cross(N+1+i, perm[i], ((signs >> 2*i) & 0b11)) for i in range(N)]
            cs = [Cycle([x-1 for x in tup], 2*N) for tup in pd]
            pd_perm = reduce((lambda x,y: x*y), cs)
            num_of_orbits = len(cycle_decomp(pd_perm))
            if num_of_orbits - N == 2:
                hfk = pd_to_hfk(pd)
                if hfk['fibered'] and hfk['seifert_genus'] > 0:
                    hfk_data.append({'pd': pd, 'hfk': hfk})
                    
hfk_data_slim = [hfk_data[0]]
for data in hfk_data:
    not_dup = True
    for other in hfk_data_slim:
        if data['hfk'] == other['hfk']:
            not_dup = False
            #print(f"{data}\n{other}\n")
    if not_dup:
        hfk_data_slim.append(data)
        
for data in hfk_data_slim:
#    print(f"{data['pd']}\n hfk: {data['hfk']['ranks']}, {data['hfk']['seifert_genus']}\n")
    print(f"{data['pd']}\n hfk: {data['hfk']}\n")

In [ ]:
N = 5
perm = (2,1,4,3,5)
for i in range(N+1):
    f_ind = perm.index(i)+1 if i != 0 else N+1
    l_ind = perm.index(i+1)+1 if i != N else 0
        
    print(f"{i, i+1} --> {f_ind, l_ind} --> {abs(f_ind-l_ind)-1}")

In [ ]:
#%%time

N = 6

def cross(x: int, y: int, sign: int):
    if sign not in range(4): raise(ArgumentError) # signs are 0b00, 0b01, 0b10, 0b11
    if (sign & 0b10):
        ft, lt = (x, x%(2*N)+1), (y, y%(2*N)+1)
    else:
        ft, lt = (y, y%(2*N)+1), (x, x%(2*N)+1)
    if (sign & 0b01):
        pd = (ft[0], lt[0], ft[1], lt[1])
    else:
        pd = (ft[0], lt[1], ft[1], lt[0])
    return pd    

def screen_out(perm):
    if (perm[0] == N) or (perm[N-1] == 1):
        return True
    else:
        for i in range(N+1):
            f_ind = perm.index(i)+1 if i != 0 else N+1
            l_ind = perm.index(i+1)+1 if i != N else 0
            if abs(f_ind-l_ind)%2 == 0:
                return True
    return False

def calc_hfk(perm):
    if screen_out(perm) is True:
        return None
    #---
    #print(f"NO, {perm}\n")
    hfk_data = []
    for signs in range(4**(N-1)): # dividing by 4 corresponds to giving the quatiant of the set of all signs w.r.t. reverting all signs at the same time
        pd = [cross(N+1+i, perm[i], ((signs >> 2*i) & 0b11)) for i in range(N)]
        cs = [Cycle([x-1 for x in tup], 2*N) for tup in pd]
        pd_perm = reduce((lambda x,y: x*y), cs) #; print(f"{pd_perm}\n\n")
        num_of_orbits = len(cycle_decomp(pd_perm))
        if num_of_orbits - N == 2:
            hfk = pd_to_hfk(pd)
            if hfk['fibered'] and hfk['seifert_genus'] > 0:
                hfk_data.append({'pd': pd, 'hfk': hfk})
    return hfk_data

hfk_data = []
fct = math.factorial(N)
with Pool(processes=8) as pool:
    output = list(pool.imap(calc_hfk, permutations(range(1,N+1), N)))
#    output = list(tqdm(pool.imap(calc_hfk, permutations(range(1, N+1), N)),
#                       total=fct, desc='hfk calc'))
    for v in output:
        if v:
            hfk_data += v

hfk_data_slim = [hfk_data[0]]
for data in hfk_data:
    not_dup = True
    for other in hfk_data_slim:
        if data['hfk'] == other['hfk']:
            not_dup = False
            #print(f"{data}\n{other}\n")
    if not_dup:
        hfk_data_slim.append(data)
        
for data in hfk_data_slim:
#    print(f"{data['pd']}\n hfk: {data['hfk']['ranks']}, {data['hfk']['seifert_genus']}\n")
    print(f"{data['pd']}\n hfk: {data['hfk']}\n")    

# Creating PD-codes of knot diagrams with given crossing number

In [2]:
def cross(x: int, y: int, sign: int, N) -> tuple:
    if sign not in range(4): raise(ArgumentError) # signs are 0b00, 0b01, 0b10, 0b11
    if (sign & 0b10):
        ft, lt = (x, x%(2*N)+1), (y, y%(2*N)+1)
    else:
        ft, lt = (y, y%(2*N)+1), (x, x%(2*N)+1)
    if (sign & 0b01):
        pd = (ft[0], lt[0], ft[1], lt[1])
    else:
        pd = (ft[0], lt[1], ft[1], lt[0])
    return pd    

def screen_out(dt: list, N) -> str:
    # if there is a kink.
    for p in dt:
        if abs(p[0] - p[1]) in {1, 2*N-1}:
            return 'kink'
    # if it is not prime
    def a(x):
        kk = x if x%2==1 else [v for v in dt if v[1] == x][0][0]
        return dt[int((kk-1)/2)][x%2]
    for bnds in combinations(range(1,2*N), 2):
        s = abs(bnds[0]-bnds[1])+1
        if s%2 == 0 and s >= 6 and 2*N-s >= 6:
            psi = set(range(min(bnds), max(bnds)+1))
            phi = set(map(a, psi))
            if psi == phi:
                # print(dt, psi, phi)
                return 'composite'
    #---
    # print(dt)
    return None

def get_DT(perm: list, N: int) -> tuple:
    dt = [(2*k+1, 2*perm[k]) for k in range(N)]
    #---
    sc = screen_out(dt, N)
    if sc: return dt, sc
    #---
    # - planarity check -
    for signs in range(2**N):
        slist = [2+(signs >> i & 1) for i in range(N)]
        pd = [cross(dt[i][0], dt[i][1], slist[i], N) for i in range(N)]
        cs = [Cycle([x-1 for x in tup], 2*N) for tup in pd]
        pm = reduce((lambda x,y: x*y), cs)
#        print(signs, slist, pd, cycle_decomp(pm))
        if len(cycle_decomp(pm)) - N == 2:
            return (dt, signs), 'realizable'
    # print(dt, pm, cycle_decomp(pm))
    return dt, 'non_planar'

In [ ]:
%%time

N = 6
counts = {'kink': 0, 'composite': 0, 'non_planar': 0, 'realizable': 0}

def f(perm):
    return get_DT(perm, N)

with Pool(processes=6) as pool:
    for v in pool.imap(f, permutations(range(1, N+1), N)):
        # print(v)
        counts[v[1]] += 1
        if v[1] == 'realizable':
            print(v[0])
        
print(counts)

In [3]:
def get_PDs(dt: list, signs: int, N: int) -> list:
    pd_list = []
    for ud in range(1, 2**(N-1)):
        pd = [cross(dt[i][0], dt[i][1], ((ud>>i&1)<<1) + (signs>>i&1), N) for i in range(N)]
        pd_list.append(pd)
    return pd_list

In [4]:
def calc_hfk(perm) -> dict:
    N = len(perm)
    output = get_DT(perm, N)
    if output[1] == 'realizable':
        dt, signs = output[0]
    else:
        return None
    pd_list = get_PDs(dt, signs, N)
    lmd_dict = {}
    for pd in pd_list:
        hfk = pd_to_hfk(pd)
        sg = hfk['seifert_genus']
        if hfk['fibered'] and sg > 0:
            ranks = hfk['ranks']
            # print(ranks, sg)
            for pg in ranks.keys():
                if pg[0] == sg:
                    lmd = pg[1]
                # elif pg[0] == (-1)*sg:
                #     lmd = (-1)*pg[1]
            mlmd = 2*sg-lmd
            if lmd < 0 or mlmd < 0:
                print(pd)
            else:
                lmd_dict.setdefault(lmd, 0)
                lmd_dict[lmd] += 1
                lmd_dict.setdefault(mlmd, 0)
                lmd_dict[mlmd] += 1
                # print(pd, hfk)
    return lmd_dict 

In [ ]:
%%time

N = 23
COUNTS = {'kink': 0, 'composite': 0, 'non_planar': 0, 'realizable': 0}

lmds = {}
with Pool(processes=6) as pool:
    for lmd_dict in tqdm(pool.imap(calc_hfk, permutations(range(1, N+1), N)), total=math.factorial(N)):
        if lmd_dict:
            COUNTS['realizable'] += 1
            for lmd in lmd_dict.keys():
                lmds.setdefault(lmd, 0)
                lmds[lmd] += lmd_dict[lmd]
        if COUNTS['realizable'] == 3:
            break

lmds_sorted = sorted(lmds.items(), key=lambda x:x[0])
print(lmds_sorted)

  0%|          | 0/25852016738884976640000 [00:00<?, ?it/s]

# scratch

In [ ]:
%%time

N = 6

with Pool(processes=6) as pool:
    output = list(pool.imap(calc_hfk, 
                            tqdm(
                                permutations(range(1, N+1), N),
                            total=math.factorial(N))
                           ))

lmd_dict_merged = {}
for dic in output:
    if dic:
        for lmd in dic.keys():
            lmd_dict_merged.setdefault(lmd, 0)
            lmd_dict_merged[lmd] += dic[lmd]
            
print(lmd_dict_merged)

{1: 12, 0: 28, 2: 28, 3: 4, 4: 8}

805 ms ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)